# Particle Packing Surrogate Model

Here, we train a surrogate model for the particle packing simulations. We capture the
presence of failed simulations, the packing fractions for two different algorithms, and
the corresponding runtimes.

In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GroupKFold
import pandas as pd
import numpy as np
import joblib
from os import path
import json

# attempted use of skl2onnx to convert to onnx failing due to protobuf error
# https://github.com/onnx/onnx/issues/4469

# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType


In [2]:
dummy = False

data_dir = path.join("..", "..", "data", "processed", "particle_packing")
model_dir = path.join("..", "..", "models", "particle_packing")

if dummy:
    model_dir = path.join(model_dir, "dummy")

cv_model_dir = path.join(model_dir, "cv")


## Import Data

In [3]:
sobol_filter = pd.read_csv(path.join(data_dir, "sobol_probability_filter.csv"))
sobol_reg = pd.read_csv(path.join(data_dir, "sobol_regression.csv"))

if dummy:
    sobol_filter = sobol_filter.head(100)
    sobol_reg = sobol_reg.head(100)


## define f(x) to calc mae scores 

In [4]:
# group kfold split for cv; addressing data leakage by using groups
def rfr_group_mae(X_array, y_array, group_array, model_name_stem, random_state=13):
    kf = GroupKFold(n_splits=5)
    mae_scores = []
    y_preds = []
    y_trues = []
    for i, (train_index, test_index) in enumerate(
        kf.split(X_array, y_array, group_array)
    ):
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = y_array[train_index], y_array[test_index]
        y_test = y_test.tolist()

        model = RandomForestRegressor(random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test).tolist()

        y_preds.append(y_pred)
        y_trues.append(y_test)
        mae = mean_absolute_error(y_test, y_pred)
        mae_scores.append(mae)
        # save model as .pkl with compression
        # https://stackoverflow.com/a/47062881/13697228
        joblib.dump(model, f"{model_name_stem}_{i}.pkl", compress=3)
    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)
    print(f"MAE for {path.basename(model_name_stem)}: {avg_mae:.4f} +/- {std_mae:.4f}")
    results = {"mae": mae_scores, "y_pred": y_preds, "y_true": y_trues}
    return results


## Features

In [5]:
common_features = [
    "mu1_div_mu3",
    "mu2_div_mu3",
    "std1",
    "std2",
    "std3",
    "comp1",
    "comp2",
    "num_particles",
    "safety_factor",
]

fba_isna_prob_features = common_features
ls_isna_prob_features = common_features
fba_features = common_features + ["fba_rank"]
ls_features = common_features + ["ls_rank"]
fba_time_s_features = common_features + ["fba_time_s_rank"]
ls_time_s_features = common_features + ["ls_time_s_rank"]


## Probability Filter
### fba

In [6]:
# create groups for Probablity filter using features of sobol_filter
sobol_filter_groups = np.array(
    [
        f"{mu1},{mu2},{std1},{std2},{std3},{comp1},{comp2},{num_part},{safety_fac}"
        for mu1, mu2, std1, std2, std3, comp1, comp2, num_part, safety_fac in zip(
            sobol_filter["mu1_div_mu3"],
            sobol_filter["mu2_div_mu3"],
            sobol_filter["std1"],
            sobol_filter["std2"],
            sobol_filter["std3"],
            sobol_filter["comp1"],
            sobol_filter["comp2"],
            sobol_filter["num_particles"],
            sobol_filter["safety_factor"],
        )
    ]
)


In [7]:
## Create a GroupKFold cross-validation iterator

X_array_fba_isna_prob = sobol_filter[fba_isna_prob_features].to_numpy()
y_array_fba_isna_prob = sobol_filter[["fba_isna_prob"]].to_numpy().ravel()

## This is the trained model on As a function of mu1_div_mu3, mu2_div_mu3, std1, std2,
## std3, comp1, comp2, num_particles, safety_factor
## label data = fba_isna_prob

fba_isna_model_stem = path.join(cv_model_dir, "spf_fba_isna_prob")
fba_isna_results = rfr_group_mae(
    X_array_fba_isna_prob,
    y_array_fba_isna_prob,
    sobol_filter_groups,
    fba_isna_model_stem,
)


MAE for spf_fba_isna_prob: 0.0490 +/- 0.0004


test loading the pickled model

In [8]:
test_data = X_array_fba_isna_prob[:5]
for i in range(5):
    model = joblib.load(f"{fba_isna_model_stem}_{i}.pkl")
    print(f"{i}: {model.predict(test_data)}")


0: [0.15800239 0.         0.00554545 0.06155267 0.1096148 ]
1: [0.16549081 0.         0.01188034 0.06469852 0.16106888]
2: [0.16487179 0.         0.0109864  0.05001859 0.08853974]
3: [0.16212352 0.00166667 0.03510412 0.06572747 0.10368279]
4: [0.15947577 0.00524242 0.01400913 0.05925311 0.08923135]


### ls

In [9]:
sobolPF_ls_isna_prob = sobol_filter[ls_isna_prob_features]
ls_isna_prob = sobol_filter[["ls_isna_prob"]]

X_array_ls_isna_prob = sobolPF_ls_isna_prob.to_numpy()
y_array_ls_isna_prob = ls_isna_prob.to_numpy().ravel()

ls_isna_model_stem = path.join(cv_model_dir, "spf_ls_isna_prob")
ls_isna_results = rfr_group_mae(
    X_array_ls_isna_prob, y_array_ls_isna_prob, sobol_filter_groups, ls_isna_model_stem
)


MAE for spf_ls_isna_prob: 0.0912 +/- 0.0004


## Packing Fraction Models
### fba

In [10]:
sobol_reg_fba = sobol_reg.dropna(subset=["fba"])
X_array_fba = sobol_reg_fba[fba_features].to_numpy()
y_array_fba = sobol_reg_fba["fba"].to_numpy().ravel()

## create group for sobol regression fba features
sobol_reg_fba_group = np.array(
    [
        f"{mu1},{mu2},{std1},{std2},{std3},{comp1},{comp2},{num_part},{safety_fac}"
        for mu1, mu2, std1, std2, std3, comp1, comp2, num_part, safety_fac in zip(
            sobol_reg_fba["mu1_div_mu3"],
            sobol_reg_fba["mu2_div_mu3"],
            sobol_reg_fba["std1"],
            sobol_reg_fba["std2"],
            sobol_reg_fba["std3"],
            sobol_reg_fba["comp1"],
            sobol_reg_fba["comp2"],
            sobol_reg_fba["num_particles"],
            sobol_reg_fba["safety_factor"],
        )
    ]
)


## GroupKFold split for cv; using groups
fba_model_stem = path.join(cv_model_dir, "sobol_reg_fba")
fba_results = rfr_group_mae(
    X_array_fba, y_array_fba, sobol_reg_fba_group, fba_model_stem
)


### ls

In [ ]:
sobol_reg_ls = sobol_reg.dropna(subset=["ls"])
X_array_ls = sobol_reg_ls[ls_features].to_numpy()
y_array_ls = sobol_reg_ls["ls"].to_numpy().ravel()

## create group for sobol regression ls features
sobol_reg_ls_group = np.array(
    [
        f"{mu1},{mu2},{std1},{std2},{std3},{comp1},{comp2},{num_part},{safety_fac}"
        for mu1, mu2, std1, std2, std3, comp1, comp2, num_part, safety_fac in zip(
            sobol_reg_ls["mu1_div_mu3"],
            sobol_reg_ls["mu2_div_mu3"],
            sobol_reg_ls["std1"],
            sobol_reg_ls["std2"],
            sobol_reg_ls["std3"],
            sobol_reg_ls["comp1"],
            sobol_reg_ls["comp2"],
            sobol_reg_ls["num_particles"],
            sobol_reg_ls["safety_factor"],
        )
    ]
)


## GroupKFold split for cv; using groups
ls_model_path = path.join(cv_model_dir, "sobol_reg_ls")
ls_results = rfr_group_mae(X_array_ls, y_array_ls, sobol_reg_ls_group, ls_model_path)


MAE for sobol_reg_ls: 0.0173 +/- 0.0048


## Runtime Models
No NaNs in the time values.
### fba_time_s

In [ ]:
## create fba_time_s dataframe to use for groups
fba_time_s_df = sobol_reg[fba_time_s_features]

X_array_fba_time_s = sobol_reg[fba_time_s_features].to_numpy()
fba_time_s = sobol_reg[["fba_time_s"]]
y_array_fba_time_s = fba_time_s.to_numpy().ravel()


##create groups for fba_time_s GroupKFOld split
sobol_reg_fba_time_s_group = np.array(
    [
        f"{mu1},{mu2},{std1},{std2},{std3},{comp1},{comp2},{num_part},{safety_fac}"
        for mu1, mu2, std1, std2, std3, comp1, comp2, num_part, safety_fac in zip(
            fba_time_s_df["mu1_div_mu3"],
            fba_time_s_df["mu2_div_mu3"],
            fba_time_s_df["std1"],
            fba_time_s_df["std2"],
            fba_time_s_df["std3"],
            fba_time_s_df["comp1"],
            fba_time_s_df["comp2"],
            fba_time_s_df["num_particles"],
            fba_time_s_df["safety_factor"],
        )
    ]
)


fba_time_s_model_stem = path.join(cv_model_dir, "sobol_reg_fba_time_s")
fba_time_s_results = rfr_group_mae(
    X_array_fba_time_s,
    y_array_fba_time_s,
    sobol_reg_fba_time_s_group,
    fba_time_s_model_stem,
)


MAE for sobol_reg_fba_time_s: 0.0367 +/- 0.0107


### ls_time_s

In [ ]:
##create df for ls_time_s
ls_time_s_df = sobol_reg[ls_time_s_features]

##create arrays for model
X_array_ls_time_s = sobol_reg[ls_time_s_features].to_numpy()
ls_time_s = sobol_reg[["ls_time_s"]]
y_array_ls_time_s = ls_time_s.to_numpy().ravel()


##create groups for fba_time_s GroupKFOld split
sobol_reg_ls_time_s_group = np.array(
    [
        f"{mu1},{mu2},{std1},{std2},{std3},{comp1},{comp2},{num_part},{safety_fac}"
        for mu1, mu2, std1, std2, std3, comp1, comp2, num_part, safety_fac in zip(
            ls_time_s_df["mu1_div_mu3"],
            ls_time_s_df["mu2_div_mu3"],
            ls_time_s_df["std1"],
            ls_time_s_df["std2"],
            ls_time_s_df["std3"],
            ls_time_s_df["comp1"],
            ls_time_s_df["comp2"],
            ls_time_s_df["num_particles"],
            ls_time_s_df["safety_factor"],
        )
    ]
)


ls_time_s_model_stem = path.join(cv_model_dir, "sobol_reg_ls_time_s")
ls_time_s_results = rfr_group_mae(
    X_array_ls_time_s,
    y_array_ls_time_s,
    sobol_reg_ls_time_s_group,
    ls_time_s_model_stem,
)


MAE for sobol_reg_ls_time_s: 2.3358 +/- 0.4001


In [ ]:
# reminder where is the data and what is it saving
main_results = {
    "fba_isna_prob": fba_isna_results,
    "ls_isna_prob": ls_isna_results,
    "fba": fba_results,
    "ls": ls_results,
    "fba_time_s": fba_time_s_results,
    "ls_time_s": ls_time_s_results,
}
with open(path.join(data_dir, "model_metadata.json"), "w") as f:
    json.dump(main_results, f)


In [ ]:
model_paths = {
    "fba_isna_prob": fba_isna_model_stem,
    "ls_isna_prob": ls_isna_model_stem,
    "fba": fba_model_stem,
    "ls": ls_model_path,
    "fba_time_s": fba_time_s_model_stem,
    "ls_time_s": ls_time_s_model_stem,
}

for i in range(5):
    models = {}
    for key, model_path in model_paths.items():
        models[key] = joblib.load(f"{model_path}_{i}.pkl")

    with open(path.join(cv_model_dir, f"cross_validation_models_{i}.pkl"), "wb") as f:
        joblib.dump(models, f, compress=3)


## Production models (full training data)
Six keys in the dictionary, each key is a value of a label, and its value pair is the trained model.
This trained model is stored in the models folder with the pickle file name "trained_model.pkl"

In [ ]:
def train_and_save(
    spf_feat_array,
    sr_feat_array,
    spf_labels_array,
    sr_labels_array,
    spf_label_names,
    sr_label_names,
):
    models = {}

    for X1, y1, name1 in zip(spf_feat_array, spf_labels_array, spf_label_names):
        print(f"X1 spf shape: {X1.shape}, Y1 spf shape: {y1.shape}")
        model = RandomForestRegressor(random_state=13)
        model.fit(X1, y1)
        models[name1] = model

    for X2, y2, name2 in zip(sr_feat_array, sr_labels_array, sr_label_names):
        print(f"X2 sr shape: {X2.shape}, Y2 sr shape: {y2.shape}")
        model = RandomForestRegressor(random_state=13)
        model.fit(X2, y2)
        models[name2] = model

    return models


In [ ]:
# List of x_arrays, y_arrays, and target_names
sobol_prob_filter_arrays = [X_array_fba_isna_prob, X_array_ls_isna_prob]
sobol_prob_filter_labels = [y_array_fba_isna_prob, y_array_ls_isna_prob]
sobol_filter_target_names = ["fba_isna_prob", "ls_isna_prob"]

# List of x_arrays, y_arrays, and target_names
sobol_reg_x_arrays = [X_array_fba, X_array_ls, X_array_fba_time_s, X_array_ls_time_s]
sobol_reg_labels = [y_array_fba, y_array_ls, y_array_fba_time_s, y_array_ls_time_s]
sobol_reg_target_names = ["fba", "ls", "fba_time_s", "ls_time_s"]

# Train and save the model on all the data
models = train_and_save(
    sobol_prob_filter_arrays,
    sobol_reg_x_arrays,
    sobol_prob_filter_labels,
    sobol_reg_labels,
    sobol_filter_target_names,
    sobol_reg_target_names,
)

joblib.dump(models, path.join(model_dir, "surrogate_models.pkl"), compress=3)


X1 spf shape: (100, 9), Y1 spf shape: (100,)
X1 spf shape: (100, 9), Y1 spf shape: (100,)
X2 sr shape: (86, 10), Y2 sr shape: (86,)
X2 sr shape: (60, 10), Y2 sr shape: (60,)
X2 sr shape: (100, 10), Y2 sr shape: (100,)
X2 sr shape: (100, 10), Y2 sr shape: (100,)


['..\\..\\models\\particle_packing\\dummy\\surrogate_models.pkl']

In [ ]:
## Code Graveyard 

In [ ]:
# -------------------------#

# original kfold split for cv; not using groups

# argument for rfr_mae, X_array, y_array, model_name to save model as .pkl
# def rfr_mae(X_array, y_array, model_name_stem, random_state=13):
#     kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
#     mae_scores = []
#     y_preds = []
#     y_trues = []
#     for i, (train_index, test_index) in enumerate(kf.split(X_array)):
#         X_train, X_test = X_array[train_index], X_array[test_index]
#         y_train, y_test = y_array[train_index], y_array[test_index]
#         y_test = y_test.tolist()

#         model = RandomForestRegressor(random_state=random_state)
#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_test).tolist()

#         y_preds.append(y_pred)
#         y_trues.append(y_test)
#         mae = mean_absolute_error(y_test, y_pred)
#         mae_scores.append(mae)
#         # save model as .pkl with compression
#         # https://stackoverflow.com/a/47062881/13697228
#         joblib.dump(model, f"{model_name_stem}_{i}.pkl", compress=3)
#     avg_mae = np.mean(mae_scores)
#     std_mae = np.std(mae_scores)
#     print(f"MAE for {path.basename(model_name_stem)}: {avg_mae:.4f} +/- {std_mae:.4f}")
#     results = {"mae": mae_scores, "y_pred": y_preds, "y_true": y_trues}
#     return results


In [ ]:
## KFold split for cv; not using groups
# fba_isna_model_stem = path.join(cv_model_dir, "spf_fba_isna_prob")
# fba_isna_results = rfr_mae(
#     X_array_fba_isna_prob, y_array_fba_isna_prob,fba_isna_model_stem
# )


In [ ]:
## KFold split for cv; not using groups
# fba_model_stem = path.join(cv_model_dir, "sobol_reg_fba")
# fba_results = rfr_mae(X_array_fba, y_array_fba, fba_model_stem)


In [ ]:
## KFold split for cv; not using groups
# ls_model_path = path.join(cv_model_dir, "sobol_reg_ls")
# ls_results = rfr_mae(X_array_ls, y_array_ls, ls_model_path)


In [76]:
# #Sobol_reg feature and target lists
# sobol_reg_features_lst = ['mu1_div_mu3', 'mu2_div_mu3', 'std1', 'std2', 'std3', 'comp1', 'comp2', 'num_particles', 'safety_factor', 'fba_rank', 'ls_rank', 'fba_time_s_rank', 'ls_time_s_rank']
# sobol_reg_target_lst = ['fba', 'ls', 'fba_time_s', 'ls_time_s']

# #Sobol_filter feature and target lists
# sobol_filter_features_lst = ['mu1_div_mu3', 'mu2_div_mu3', 'std1', 'std2', 'std3', 'comp1', 'comp2', 'num_particles', 'safety_factor']
# sobol_filter_target_lst = ['fba_isna_prob', 'ls_isna_prob']


# def train_and_save(df1, df2,feature_lst_df1,feature_lst_df2,label_lst_df1, label_lst_df2):
#     models = {}

#     for label in label_lst_df1:
#         X = df1[feature_lst_df1]
#         y = df1[label]

#         model = RandomForestRegressor(random_state=13)
#         model.fit(X, y)
#         model.predict(X)

#         models[label] = model

#     for label in label_lst_df2:
#         X = df2[feature_lst_df2]
#         y = df2[label]

#         model = RandomForestRegressor(random_state=13)
#         model.fit(X, y)

#         models[label] = model

#     joblib.dump(models, "trained_model.pkl")
#     return models


In [ ]:
# get list of arrays of x_arrays and y_arrays
# X_arrays = [
#     X_array_fba_isna_prob,
#     X_array_ls_isna_prob,
#     X_array_fba,
#     X_array_ls,
#     X_array_ls_time_s,
#     X_array_ls_time_s,
# ]
# y_arrays = [
#     y_array_fba_isna_prob,
#     y_array_ls_isna_prob,
#     y_array_fba,
#     y_array_ls,
#     y_array_fba_time_s,
#     y_array_ls_time_s,
# ]


# rf = RandomForestRegressor(random_state=13)
# trained_model_fba_isna_prob = rf.fit(X_array_fba_isna_prob, y_array_fba_isna_prob)
# with open(path.join(model_dir, "trained_model_fba_isna_prob.pkl"), "wb") as f:
#     joblib.dump(trained_model_fba_isna_prob, f)


In [ ]:
# print("Average MAE for fba_isna_prob",rfr_mae(X_array_fba_isna_prob, y_array_fba_isna_prob,'fba_isna_prob.pkl'))

# load trained model
# loaded_model = joblib.load('fba_isna_prob_model.pkl')

# Save the model
# with open('../models/fba_isna_prob.pkl', 'wb') as f:
#     pickle.dump(model, f)

# # Load the model
# with open('path/to/save/model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)


In [ ]:
# url_sobol_filter = "https://zenodo.org/record/7513019/files/sobol_probability_filter.csv"
# sobol_filter = pd.read_csv(url_sobol_filter)

# url_sobol_reg = "https://zenodo.org/record/7513019/files/sobol_regression.csv"
# sobol_reg = pd.read_csv(url_sobol_reg)


In [ ]:
# os.getcwd()
# os.chdir("../data/raw")

# sobol_filter.to_csv('sobol_filter.csv', index=False)

# sobol_reg.to_csv('sobol_reg.csv', index=False)


In [ ]:
# read in sobol_regression.csv
# url_sobol_reg = "https://zenodo.org/record/7513019/files/sobol_regression.csv"


In [ ]:
# sobol_reg_x = sobol_reg[
#     [
#         "mu1_div_mu3",
#         "mu2_div_mu3",
#         "std1",
#         "std2",
#         "std3",
#         "comp1",
#         "comp2",
#         "num_particles",
#         "safety_factor",
#         "fba_rank",
#         "ls_rank",
#         "fba_time_s_rank",
#         "ls_time_s_rank",
#     ]
# ]


In [ ]:
# print(len(sobol_reg_x))
# print(len(fba))


In [ ]:
# print(
#     "Average MAE for ls_time_s",
#     rfr_mae(X_array_fba_time_s, y_array_ls_time_s, "sobol_reg_ls_time_s.pkl"),
# )


In [ ]:
# # parse data for target "fba_isna_prob"
# fba_isna_prob = sobol_filter["fba_isna_prob"]
# sobolPF_fba_isna_prob = sobol_filter.drop(["ls_isna_prob", "fba_isna_prob"], axis=1)
# fba_isna_prob = fba_isna_prob.to_frame()
